In [58]:
# Importar librerías necesarias

import pandas as pd
import numpy as np
import re

pd.set_option('display.max_columns', None)

from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

from datetime import datetime


In [49]:
# Importar datos

df = pd.read_csv("../data/finanzas_hotel_bookings.csv", index_col=0)

C:\Users\carla\AppData\Local\Temp\ipykernel_14532\3529265360.py:3: DtypeWarning: Columns (1,2,5,13,14,15,16,20,21,26,30,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/finanzas_hotel_bookings.csv", index_col=0)


In [24]:
# Definir una función para explorar las caraterísticas de una columna rápidamente

def quick_check(df, column_name):
        """ This function is for testing purposes, to quicky check data type and unique values of a column"""
        print (f"\nColumn name: {column_name}")
        print (f"\nData type: {df[column_name].dtype}")
        print (f"\nNot null count: {df[column_name].notnull().sum()}")
        print (f"\nNull count: {df[column_name].isnull().sum()}, {round(df[column_name].isnull().sum()/df.shape[0]*100)}%")
        print (f"\nDuplicated values: {df[column_name].duplicated().sum()}")
        print (f"\nUnique values: \n{df[column_name].unique()}")
        print (f"\nValue counts: \n{df[column_name].value_counts()}") 

In [50]:
# Eliminar filas a partir de la 119390
df = df.iloc[:119390]

# Imputación de nulos de la columna 'reservation_status_date'

In [ ]:
# Primero exploro la coumna "reservation_status" ya que me puede ayudar a completar este campo
    # - Si el reservation status es "Check-out", entonces la "reservation_status_date" es igual a "arrival_date" + "total_stays".
    # - Si el reservation status es "No-Show", entonces la "reservation_status_date" a "arrival_date".
    # - Si el reservation status es "Canceled", no es posible deducir la "reservation_status_date".

In [6]:
df['reservation_status'].unique()

array(['Check-Out', 'Canceled', 'No-Show'], dtype=object)

In [64]:
# ¿Cuántos registros tienen "reservation_status_date" nulo?

df[df['reservation_status_date'].isnull()].shape[0]

12891

In [17]:
# ¿Cuántos registros con reservation status "Check-out" tienen "reservation_status_date" nulo?

df[(df['reservation_status'] == 'Check-Out') & (df['reservation_status_date'].isnull())].shape[0]

8097

In [18]:
# ¿Cuántos registros con reservation status "No-show" tienen "reservation_status_date" nulo?

df[(df['reservation_status'] == 'No-Show') & (df['reservation_status_date'].isnull())].shape[0] 

127

In [62]:
# ¿Cuántos registros con reservation status "Canceled" tienen "reservation_status_date" nulo?

df[(df['reservation_status'] == 'Canceled') & (df['reservation_status_date'].isnull())].shape[0] 

4667

In [20]:
# ¿Cuántos registros con reservation status "Canceled" tienen "reservation_status_date" NO nulo? --> Estos son los que voy a usar para imputar por la mediana

df[(df['reservation_status'] == 'Canceled') & (df['reservation_status_date'].notnull())].shape[0] 

38350

In [ ]:
# Imputación de nulos de la columna "reservation_status_date" --> Dos partes.

# 1) Imputación a partir de la fecha de arrival (que es una columna que debemos crear).
    # Crear columna 'arrival_date'    
    # Crear un condicional a partir del "reservation_status" (esto ya lo creó Laura) 
        # Si el reservation status es "Check-out", entonces la "reservation_status_date" es igual a "arrival_date" + "total_stays".
        # Si el reservation status es "No-Show", entonces la "reservation_status_date" a "arrival_date".
        # Si el reservation status es "Canceled", no es posible deducir la "reservation_status_date" a partir de la fecha de arrival, continúa en Parte 2.

# 2) Imputación a partir del tiempo de cancelación.
    # Crear la columna "reservation_date" a partir de la diferencia entre la "arrival_date" y el "lead_time" (esto ya lo creó Laura)            
    # Crear una nueva columna llamada "cancelation_time" que se calcula como la diferencia entre la fecha de cancelación y la fecha de la reserva
    # Imputar los nulos que pueda haber en la columna creada ("cancelation_time") con la mediana
    # Imputar los nulos pendientes de la columna "reservation_status_date" a partir de "cancelation_time"

## 1) Imputación a partir de la fecha de arrival

###  Creación columna "arrival_date"

In [53]:
# Verificar el tipo de dato de cada columna
print(f"La columna 'arrival_date_year' es del tipo {df['arrival_date_year'].dtype}")
print(f"La columna 'arrival_date_month' es del tipo {df['arrival_date_month'].dtype}")
print(f"La columna 'arrival_date_day_of_month' es del tipo {df['arrival_date_day_of_month'].dtype}")

La columna 'arrival_date_year' es del tipo float64
La columna 'arrival_date_month' es del tipo object
La columna 'arrival_date_day_of_month' es del tipo float64


In [54]:
# Cambiar el formato de la columna 'arrival_date_month' a numérico 

# Diccionario de mapeo de meses y valores numéricos
dic_map = {
    'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 
    'June': 6, 'July': 7, 'August': 8, 'September': 9, 'October': 10, 
    'November': 11, 'December': 12, 
    1: 1, 2: 2, 3: 3, 4:4, 5:5, 6:6, 7:7, 8:8, 9:9, 10:10, 11:11, 12:12, 'nan': np.nan}

# Asegurarse de que los valores en la columna sean de tipo string para mapear correctamente
df['arrival_date_month'] = df['arrival_date_month'].astype(str).map(dic_map).astype('Int64')

# Verificación
print(f"La columna 'arrival_date_month' es del tipo {df['arrival_date_month'].dtype}")

La columna 'arrival_date_month' es del tipo Int64


In [ ]:
# Crear la columna "arrival_date" --> ⚠️ Esto no me funciona.

# Rellenar los valores nulos con '01' para día y mes y un valor de año predeterminado (ej. '2000')
df['arrival_date_year'] = df['arrival_date_year'].fillna(2000).astype(int)
df['arrival_date_month'] = df['arrival_date_month'].fillna(1).astype(int)
df['arrival_date_day_of_month'] = df['arrival_date_day_of_month'].fillna(1).astype(int)

# Crear la columna de fecha
df['arrival_date'] = pd.to_datetime(df[['arrival_date_year', 'arrival_date_month', 'arrival_date_day_of_month']].astype(str).agg('-'.join, axis=1), 
                             format='%Y-%m-%d', errors='coerce')

# Si quieres restaurar los valores originales nulos a NaT en la columna 'fecha'
df.loc[df[['arrival_date_year', 'arrival_date_month', 'arrival_date_day_of_month']].isnull().any(axis=1), 'arrival_date'] = pd.NaT

# Ver el DataFrame con la nueva columna 'fecha'
print(df[['arrival_date_year', 'arrival_date_month', 'arrival_date_day_of_month', 'arrival_date']])

In [45]:
# Este código comentado es el código que está en la función de Laura para crear la columna "arrival_date" el cual tampoco me funciona.
# df['arrival_date'] = pd.to_datetime(df[['arrival_date_year', 'arrival_date_month', 'arrival_date_day_of_month']].astype(str).agg('-'.join, axis=1), format='%Y-%m-%d', errors='coerce')

In [56]:
quick_check(df,'arrival_date')


Column name: arrival_date

Data type: datetime64[ns]

Not null count: 119390

Null count: 0, 0%

Duplicated values: 118231

Unique values: 
<DatetimeArray>
['2015-07-01 00:00:00', '2000-07-01 00:00:00', '2015-07-02 00:00:00',
 '2000-07-02 00:00:00', '2000-07-03 00:00:00', '2015-07-03 00:00:00',
 '2015-07-04 00:00:00', '2000-07-04 00:00:00', '2015-07-05 00:00:00',
 '2000-07-05 00:00:00',
 ...
 '2015-12-13 00:00:00', '2016-03-06 00:00:00', '2000-03-12 00:00:00',
 '2016-03-26 00:00:00', '2016-11-13 00:00:00', '2016-12-04 00:00:00',
 '2016-12-18 00:00:00', '2016-12-25 00:00:00', '2017-02-27 00:00:00',
 '2017-03-17 00:00:00']
Length: 1159, dtype: datetime64[ns]

Value counts: 
arrival_date
2000-10-16    306
2000-08-14    295
2000-01-02    291
2000-06-17    274
2000-01-19    271
             ... 
2015-12-07     14
2015-12-15     14
2015-07-28     13
2015-11-29     10
2015-12-13      8
Name: count, Length: 1159, dtype: int64


### Crear un condicional a partir del "reservation_status"

# ANTERIOR -------------------------------------------------------------

In [4]:
# Definir una función para explorar las caraterísticas de una columna rápidamente

def quick_check(df, column_name):
        """ This function is for testing purposes, to quicky check data type and unique values of a column"""
        print (f"\nColumn name: {column_name}")
        print (f"\nData type: {df[column_name].dtype}")
        print (f"\nNot null count: {df[column_name].notnull().sum()}")
        print (f"\nNull count: {df[column_name].isnull().sum()}, {round(df[column_name].isnull().sum()/df.shape[0]*100)}%")
        print (f"\nDuplicated values: {df[column_name].duplicated().sum()}")
        print (f"\nUnique values: \n{df[column_name].unique()}")
        print (f"\nValue counts: \n{df[column_name].value_counts()}") 

# Consigna 1 - Transformaciones en la columna 'reservation_status_date'

1. Reemplazar las fechas erróneas (aquellas con formato 'AAAA-MM-DD') por nulos 
2. Imputar todos los nulos con KNN
3. Quitar la componente 'HH:MM:SS' (al ser todas 00:00:00 no aportan valor) --> Antes de hacerlo, verificar cómo lo hizo Laura, para que quede alineado con las fechas que trabajó ella. ❗❗ 
4. Cambiar el formato de la columna a "datetime" (usar to_datetime)

In [6]:
# 1) Definir función para reemplazar las fechas erróneas aquellas con formato 'AAAA-MM-DD' por nulos

def reemplazar_fechas_erroneas(fecha):
    """
    Dada una fecha, esta función verifica si el formato de la misma coincide con 'AAAA-MM-DD' y de ser así devuelve un valor nulo.   
    Si el formato no coincide o si no es cadena de texto devuelve la misma fecha.

    Args:
        fecha (string): fecha cuyo formato se quiere verificar.

    Returns: 
        np.null: si el formato coincide.
        El mismo argumento que se le ingresó si el formato no coincide o si no es cadena de texto. 
    """
    # Verificar si el valor es una cadena de texto
    if isinstance(fecha, str):
        # Expresión regular para identificar fechas en formato 'AAAA-MM-DD'
        patron = r'^\d{4}-\d{2}-\d{2}$'
        if re.match(patron, fecha):
            return np.nan
    # Si no es una cadena de texto o no coincide con el patrón, devolver el valor original
    return fecha


In [70]:
# Chequeo previo: Reviso los nulos de las columnas antes de aplicarles la función
quick_check(df,'reservation_status_date')


Column name: reservation_status_date

Data type: object

Not null count: 106514

Null count: 76363, 42%

Duplicated values: 181885

Unique values: 
['2015-07-01 00:00:00' '2015-07-02 00:00:00' '2015-07-03 00:00:00'
 '2015-05-06 00:00:00' '2015-04-22 00:00:00' '2015-06-23 00:00:00'
 '2015-07-05 00:00:00' nan '2015-07-07 00:00:00' '2015-07-08 00:00:00'
 '2015-05-11 00:00:00' '2015-07-15 00:00:00' '2015-07-16 00:00:00'
 '2015-05-29 00:00:00' '2015-05-19 00:00:00' '2015-06-19 00:00:00'
 '2015-07-06 00:00:00' '2015-05-23 00:00:00' '2015-05-18 00:00:00'
 '2015-07-09 00:00:00' '2015-06-02 00:00:00' '2015-07-04 00:00:00'
 '2015-06-29 00:00:00' '2015-06-16 00:00:00' '2015-06-18 00:00:00'
 '2015-06-12 00:00:00' '2015-06-09 00:00:00' '2015-05-26 00:00:00'
 '2015-07-11 00:00:00' '2015-07-12 00:00:00' '2015-07-13 00:00:00'
 '2015-07-17 00:00:00' '2015-04-15 00:00:00' '2015-05-13 00:00:00'
 '2015-07-10 00:00:00' '2015-05-20 00:00:00' '2015-05-12 00:00:00'
 '2015-07-14 00:00:00' '2015-06-17 00:00:00

In [7]:
# Llamo a la función

column = 'reservation_status_date'
df[column] = df[column].apply(reemplazar_fechas_erroneas)

In [8]:
# Chequeo porterior: Reviso los nulos de las columnas luego de aplicarles la función para verificar que los nulos hayan sido imputados 

quick_check(df,column)


Column name: reservation_status_date

Data type: object

Not null count: 106469

Null count: 76408, 42%

Duplicated values: 181930

Unique values: 
['2015-07-01 00:00:00' '2015-07-02 00:00:00' '2015-07-03 00:00:00'
 '2015-05-06 00:00:00' '2015-04-22 00:00:00' '2015-06-23 00:00:00'
 '2015-07-05 00:00:00' nan '2015-07-07 00:00:00' '2015-07-08 00:00:00'
 '2015-05-11 00:00:00' '2015-07-15 00:00:00' '2015-07-16 00:00:00'
 '2015-05-29 00:00:00' '2015-05-19 00:00:00' '2015-06-19 00:00:00'
 '2015-07-06 00:00:00' '2015-05-23 00:00:00' '2015-05-18 00:00:00'
 '2015-07-09 00:00:00' '2015-06-02 00:00:00' '2015-07-04 00:00:00'
 '2015-06-29 00:00:00' '2015-06-16 00:00:00' '2015-06-18 00:00:00'
 '2015-06-12 00:00:00' '2015-06-09 00:00:00' '2015-05-26 00:00:00'
 '2015-07-11 00:00:00' '2015-07-12 00:00:00' '2015-07-13 00:00:00'
 '2015-07-17 00:00:00' '2015-04-15 00:00:00' '2015-05-13 00:00:00'
 '2015-07-10 00:00:00' '2015-05-20 00:00:00' '2015-05-12 00:00:00'
 '2015-07-14 00:00:00' '2015-06-17 00:00:00

In [16]:
df['reservation_status_date'] = pd.to_datetime(df['reservation_status_date']) 
df['reservation_status_date'] = df['reservation_status_date'].map(pd.Timestamp.timestamp) 
# Crear el imputer KNN 
imputer = KNNImputer(n_neighbors=5) 
# Seleccionar las características para imputar 
df_imputed = imputer.fit_transform(df) 
# Convertir de vuelta a DataFrame 
df_imputed = pd.DataFrame(df_imputed, columns=df.columns) 
# Convertir los timestamps de vuelta a fechas 
df_imputed['reservation_status_date'] = pd.to_datetime(df_imputed['reservation_status_date'], unit='s') 
print(df_imputed)

TypeError: descriptor 'timestamp' for 'pandas._libs.tslibs.timestamps._Timestamp' objects doesn't apply to a 'NaTType' object

In [12]:
# # 2) Imputar nulos con KNN (usando la función de Laura) ⚠️

# def imputar_knn_fecha_reservation_status(df, n_neighbors=5):
#     # Seleccionar las columnas relevantes para la imputación
#     cols_fecha = ['reservation_status_date']
    
#     # 
#     df_fechas = pd.to_datetime(df['reservation_status_date'])
    
#     # Imputar los valores nulos usando KNN
#     imputer = KNNImputer(n_neighbors=n_neighbors)
#     df_imputado = imputer.fit_transform(df_fechas)
    
#     # Convertir el resultado de vuelta a un DataFrame con las mismas columnas
#     df_imputado = pd.DataFrame(df_imputado, columns=cols_fecha)
    
#     # Reemplazar las columnas originales en el DataFrame con los valores imputados y ajustados
#     df[cols_fecha] = df_imputado
    
#     # Imprimir los nulos restantes en las columnas relevantes
#     print(f'Nulos después de la operación:')
#     print(f'- reservation_status_date: {df["reservation_status_date"].isna().sum()}')


In [13]:
imputar_knn_fecha_reservation_status(df)

ValueError: Expected 2D array, got 1D array instead:
array=[ 1.43570880e+18  1.43570880e+18  1.43579520e+18 ... -9.22337204e+18
 -9.22337204e+18 -9.22337204e+18].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [74]:
# 3) Quitar la componente 'HH:MM:SS' (al ser todas 00:00:00 no aportan valor) 
# ❗❗ Antes de hacerlo, verificar cómo lo hizo Laura, para que quede alineado con las fechas que trabajó ella.  

# Función para transformar fechas con formato 'AAAA-MM-DD 00:00:00' en 'AAAA-MM-DD'
def quitar_horas (fecha):
    """ 
    A partir de una fecha, esta función divide los componentes de la misma por espacios y retorna el primer elemento.  
    """
    # Verificar si el valor es una cadena de texto
    if isinstance(fecha, str):
        fecha = fecha.split()[0]
        return fecha
    # Si no es una cadena de texto, devolver el valor original
    return fecha


df[column] = df[column].apply(quitar_horas)

In [75]:
# 4) Cambiar el formato a date time

df[column] = pd.to_datetime(df[column])

# Consigna 2 - Imputación de nulos por "Unknown"

- Definir una función para imputar valores nulos por una categoría llamada "Unknown".
- Esta función se aplicará en las columnas "Country", "Company" y "Agent".

In [76]:
# 1) Defino la función para imputar nulos por 'Unknown'

def imputar_por_unknown(dataframe, lista_columnas):
        """
        Esta función itera sobre una lista de columnas dentro de un dataframe y reemplaza los nulos por 'Unknown'.

        Args:
            dataframe (pandas dataframe): Es el dataframe que contiene los datos.
            lista_columnas (lista): Lista de columnas existentes en el dataframe sobre las cuales se quiere imputar los nulos.

        Returns:
            dataframe: Dataframe actualizado.
        """
        for columna in lista_columnas:
            # Verificar si la columna existe en el dataframe
            if columna in dataframe:
                # Si existe, reemplazar nulos por 'Unknown'
                dataframe[columna] = dataframe[columna].fillna("Unknown")
            else:
                # Si no existe, imprimir un mensaje de error
                print(f"❌ La columna '{columna}' no existe en el DataFrame.")

        return dataframe

In [77]:
# 2) Defino los argumentos para llamar la función: 

# Creo la lista de columnas sobre las cuales quiero aplicar la función 
lista_columnas = ['country', 'company', 'agent']

In [78]:
# 3) Chequeo previo: Reviso los nulos de las columnas antes de aplicarles la función

quick_check(df, 'country')

quick_check(df, 'company')

quick_check(df, 'agent')


Column name: country

Data type: object

Not null count: 65054

Null count: 117823, 64%

Duplicated values: 182713

Unique values: 
['PRT' nan 'GBR' 'USA' 'ESP' 'IRL' 'FRA' 'ROU' 'NOR' 'OMN' 'ARG' 'DEU'
 'CHE' 'GRC' 'NLD' 'DNK' 'RUS' 'POL' 'AUS' 'EST' 'CZE' 'BRA' 'BEL' 'CN'
 'SWE' 'FIN' 'MOZ' 'SVN' 'MAR' 'ITA' 'UKR' 'SMR' 'LVA' 'PRI' 'CHL' 'CHN'
 'LTU' 'LUX' 'AUT' 'TUR' 'MEX' 'ZAF' 'AGO' 'ISR' 'IND' 'CYM' 'ZMB' 'CPV'
 'ZWE' 'DZA' 'KOR' 'HUN' 'ARE' 'TUN' 'JAM' 'ALB' 'HRV' 'HKG' 'AND' 'GIB'
 'URY' 'BLR' 'JEY' 'CYP' 'MDV' 'FJI' 'KAZ' 'PAK' 'IDN' 'LBN' 'PHL' 'COL'
 'SEN' 'GEO' 'AZE' 'BHR' 'NZL' 'THA' 'DOM' 'MYS' 'VEN' 'ARM' 'LKA' 'CUB'
 'CMR' 'IRN' 'BIH' 'NGA' 'COM' 'BGR' 'CIV' 'SRB' 'JOR' 'SYR' 'BDI' 'SGP'
 'KWT' 'PLW' 'QAT' 'SVK' 'SUR' 'MLT' 'MWI' 'MDG' 'ISL' 'JPN' 'CAF' 'TGO'
 'TWN' 'DJI' 'VNM' 'PER' 'EGY' 'SAU' 'KNA' 'ETH' 'ECU' 'IRQ' 'KHM' 'MCO'
 'BGD' 'TJK' 'NIC' 'GGY' 'BEN' 'VGB' 'CRI' 'TZA' 'GAB' 'MKD' 'TMP' 'GLP'
 'LIE' 'GNB' 'MAC' 'IMN' 'UMI' 'MYT' 'GHA' 'FRO' 'MMR' 'PAN' 'MUS' 

In [79]:
# 4) Llamo a la función
imputar_por_unknown (df, lista_columnas)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,0
0,Resort Hotel,False,342.000000,2015.0,July,27.0,1.0,0.000000,0.000000,2.0,NaN,0.0,BB,PRT,NaN,Direct,0.0,NaN,0.0,C,C,3.0,Unknown,Unknown,0.0,Transient,0.0,0.0,0.0,Check-Out,2015-07-01,NaN
1,Resort Hotel,False,737.000000,NaN,July,27.0,1.0,0.000000,0.000000,2.0,NaN,0.0,BB,Unknown,NaN,Direct,0.0,0.0,0.0,NaN,C,4.0,Unknown,Unknown,0.0,Transient,0.0,0.0,0.0,Check-Out,2015-07-01,NaN
2,Resort Hotel,False,7.000000,2015.0,July,27.0,1.0,0.000000,1.000000,1.0,0.0,0.0,BB,GBR,NaN,Direct,0.0,0.0,0.0,A,C,0.0,Unknown,Unknown,0.0,Transient,75.0,0.0,0.0,Check-Out,2015-07-02,NaN
3,Resort Hotel,False,13.000000,NaN,July,27.0,1.0,0.000000,1.000000,1.0,NaN,0.0,BB,GBR,Corporate,Corporate,0.0,0.0,0.0,A,A,0.0,304.0,Unknown,0.0,Transient,75.0,0.0,0.0,Check-Out,2015-07-02,NaN
4,Resort Hotel,False,14.000000,NaN,July,NaN,1.0,0.000000,2.000000,2.0,NaN,0.0,BB,Unknown,Online TA,TA/TO,0.0,0.0,0.0,A,A,0.0,240.0,Unknown,0.0,Transient,98.0,0.0,1.0,Check-Out,2015-07-03,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182872,NaN,NaN,224.337762,NaN,NaN,NaN,NaN,19.005545,19.819823,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
182873,NaN,NaN,390.141963,NaN,NaN,NaN,NaN,14.751794,19.989726,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
182874,NaN,NaN,230.689826,NaN,NaN,NaN,NaN,11.409496,20.461372,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
182875,NaN,NaN,304.888534,NaN,NaN,NaN,NaN,16.744472,15.400773,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN


In [80]:
# 5) Chequeo porterior: Reviso los nulos de las columnas luego de aplicarles la función para verificar que los nulos hayan sido imputados 

quick_check(df, 'country')

quick_check(df, 'company')

quick_check(df, 'agent')


Column name: country

Data type: object

Not null count: 182877

Null count: 0, 0%

Duplicated values: 182713

Unique values: 
['PRT' 'Unknown' 'GBR' 'USA' 'ESP' 'IRL' 'FRA' 'ROU' 'NOR' 'OMN' 'ARG'
 'DEU' 'CHE' 'GRC' 'NLD' 'DNK' 'RUS' 'POL' 'AUS' 'EST' 'CZE' 'BRA' 'BEL'
 'CN' 'SWE' 'FIN' 'MOZ' 'SVN' 'MAR' 'ITA' 'UKR' 'SMR' 'LVA' 'PRI' 'CHL'
 'CHN' 'LTU' 'LUX' 'AUT' 'TUR' 'MEX' 'ZAF' 'AGO' 'ISR' 'IND' 'CYM' 'ZMB'
 'CPV' 'ZWE' 'DZA' 'KOR' 'HUN' 'ARE' 'TUN' 'JAM' 'ALB' 'HRV' 'HKG' 'AND'
 'GIB' 'URY' 'BLR' 'JEY' 'CYP' 'MDV' 'FJI' 'KAZ' 'PAK' 'IDN' 'LBN' 'PHL'
 'COL' 'SEN' 'GEO' 'AZE' 'BHR' 'NZL' 'THA' 'DOM' 'MYS' 'VEN' 'ARM' 'LKA'
 'CUB' 'CMR' 'IRN' 'BIH' 'NGA' 'COM' 'BGR' 'CIV' 'SRB' 'JOR' 'SYR' 'BDI'
 'SGP' 'KWT' 'PLW' 'QAT' 'SVK' 'SUR' 'MLT' 'MWI' 'MDG' 'ISL' 'JPN' 'CAF'
 'TGO' 'TWN' 'DJI' 'VNM' 'PER' 'EGY' 'SAU' 'KNA' 'ETH' 'ECU' 'IRQ' 'KHM'
 'MCO' 'BGD' 'TJK' 'NIC' 'GGY' 'BEN' 'VGB' 'CRI' 'TZA' 'GAB' 'MKD' 'TMP'
 'GLP' 'LIE' 'GNB' 'MAC' 'IMN' 'UMI' 'MYT' 'GHA' 'FRO' 'MMR' 'PAN' 'MUS'

# Consigna 3 - Imputación de nulos por la moda

- Definir una función para imputar valores nulos por la moda.
- Esta función se aplicará en las columnas "is_repeated_guest" y "customer_type".


In [81]:
# 1) Defino la función para imputar nulos por la moda

def imputar_por_moda (dataframe, lista_columnas):
        """
        Esta función itera sobre una lista de columnas dentro de un dataframe y reemplaza los nulos por la moda.

        Args:
            dataframe (pandas dataframe): Es el dataframe que contiene los datos.
            lista_columnas (lista): Lista de columnas existentes en el dataframe sobre las cuales se quiere imputar los nulos.

        Returns:
            dataframe: Dataframe actualizado.
        """
        for columna in lista_columnas:
            # Verificar si la columna existe en el dataframe
            if columna in dataframe.columns:
                # Si existe, calcular la moda y reemplazar los nulos con la moda
                moda = dataframe[columna].mode()[0]
                dataframe[columna] = dataframe[columna].fillna(moda)
            else:
                print(f"❌ La columna '{columna}' no existe en el DataFrame.")
        return dataframe

In [82]:
# 2) creo la lista de columnas sobre las cuales quiero aplicar la función
lista_columnas = ["is_repeated_guest","customer_type"]


In [83]:
# 3) Chequeo previo: Reviso los nulos de las columnas antes de aplicarles la función

quick_check(df, "is_repeated_guest")

quick_check(df, "customer_type")


Column name: is_repeated_guest

Data type: float64

Not null count: 114376

Null count: 68501, 37%

Duplicated values: 182874

Unique values: 
[ 0. nan  1.]

Value counts: 
0.0    110730
1.0      3646
Name: is_repeated_guest, dtype: int64

Column name: customer_type

Data type: object

Not null count: 94199

Null count: 88678, 48%

Duplicated values: 182872

Unique values: 
['Transient' nan 'Contract' 'Transient-Party' 'Group']

Value counts: 
Transient          70610
Transient-Party    19839
Contract            3290
Group                460
Name: customer_type, dtype: int64


In [84]:
# 4) Llamo a la función
imputar_por_moda (df, lista_columnas)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,0
0,Resort Hotel,False,342.000000,2015.0,July,27.0,1.0,0.000000,0.000000,2.0,NaN,0.0,BB,PRT,NaN,Direct,0.0,NaN,0.0,C,C,3.0,Unknown,Unknown,0.0,Transient,0.0,0.0,0.0,Check-Out,2015-07-01,NaN
1,Resort Hotel,False,737.000000,NaN,July,27.0,1.0,0.000000,0.000000,2.0,NaN,0.0,BB,Unknown,NaN,Direct,0.0,0.0,0.0,NaN,C,4.0,Unknown,Unknown,0.0,Transient,0.0,0.0,0.0,Check-Out,2015-07-01,NaN
2,Resort Hotel,False,7.000000,2015.0,July,27.0,1.0,0.000000,1.000000,1.0,0.0,0.0,BB,GBR,NaN,Direct,0.0,0.0,0.0,A,C,0.0,Unknown,Unknown,0.0,Transient,75.0,0.0,0.0,Check-Out,2015-07-02,NaN
3,Resort Hotel,False,13.000000,NaN,July,27.0,1.0,0.000000,1.000000,1.0,NaN,0.0,BB,GBR,Corporate,Corporate,0.0,0.0,0.0,A,A,0.0,304.0,Unknown,0.0,Transient,75.0,0.0,0.0,Check-Out,2015-07-02,NaN
4,Resort Hotel,False,14.000000,NaN,July,NaN,1.0,0.000000,2.000000,2.0,NaN,0.0,BB,Unknown,Online TA,TA/TO,0.0,0.0,0.0,A,A,0.0,240.0,Unknown,0.0,Transient,98.0,0.0,1.0,Check-Out,2015-07-03,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182872,NaN,NaN,224.337762,NaN,NaN,NaN,NaN,19.005545,19.819823,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,Unknown,Unknown,NaN,Transient,NaN,NaN,NaN,NaN,NaT,NaN
182873,NaN,NaN,390.141963,NaN,NaN,NaN,NaN,14.751794,19.989726,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,Unknown,Unknown,NaN,Transient,NaN,NaN,NaN,NaN,NaT,NaN
182874,NaN,NaN,230.689826,NaN,NaN,NaN,NaN,11.409496,20.461372,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,Unknown,Unknown,NaN,Transient,NaN,NaN,NaN,NaN,NaT,NaN
182875,NaN,NaN,304.888534,NaN,NaN,NaN,NaN,16.744472,15.400773,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,Unknown,Unknown,NaN,Transient,NaN,NaN,NaN,NaN,NaT,NaN


In [85]:
# 5) Chequeo porterior: Reviso los nulos de las columnas luego de aplicarles la función para verificar que los nulos hayan sido imputados 

quick_check(df, "is_repeated_guest")

quick_check(df, "customer_type")


Column name: is_repeated_guest

Data type: float64

Not null count: 182877

Null count: 0, 0%

Duplicated values: 182875

Unique values: 
[0. 1.]

Value counts: 
0.0    179231
1.0      3646
Name: is_repeated_guest, dtype: int64

Column name: customer_type

Data type: object

Not null count: 182877

Null count: 0, 0%

Duplicated values: 182873

Unique values: 
['Transient' 'Contract' 'Transient-Party' 'Group']

Value counts: 
Transient          159288
Transient-Party     19839
Contract             3290
Group                 460
Name: customer_type, dtype: int64
